<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/OK_Keras_Dense_Spacy_NLP_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [0]:
#USING VECTORIZATION BY SPACY NLP AND KERAS SEQUENTIAL NEURAL NETWORK FOR TRAINING ON TRAIN DATA AND MAKING PREDICTIONS
#DATASET: WHISKY RATINGS CLASSIFICATION FROM REVIEWS

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')

In [0]:
train.head(1)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1


In [0]:
train['text'] = train['description'].str.strip('\n')

In [0]:
train2 = train.drop(columns=['description', 'id'])

In [0]:
train3 = train2.rename(columns={"ratingCategory": "label"})

In [0]:
train3.head(1)

,label,text
0,1,"Sometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133"


In [0]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train3, stratify = train3['label'], test_size = 0.20, random_state=42)

df_trn.shape, df_val.shape

((3269, 2), (818, 2))

In [0]:
#Vectorize text using spacy
pip install -U spacy[cuda92]


Requirement already up-to-date: spacy[cuda92] in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [0]:
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [0]:
doc = nlp("NLP is awesome!")

In [0]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [0]:
X = df_trn['text']
Y = df_trn['label']
X_spacy = get_word_vectors(X)

In [0]:
X_train = pd.DataFrame(X_spacy)

In [0]:
X_train.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.045203,0.225353,-0.122706,-0.091795,0.05775,0.15319,-0.109985,0.000474,-0.040869,1.68712,-0.112916,0.052514,-0.097823,-0.108442,-0.096102,-0.150793,-0.020358,1.114478,-0.122863,-0.077131,-0.012964,0.062064,0.015005,-0.013178,-0.032395,-0.082518,0.064337,0.004904,0.116789,-0.303146,-0.009842,0.048855,-0.085829,-0.082199,0.124694,-0.039435,0.018605,0.034327,-0.041166,0.073664,...,0.065737,0.009891,0.081973,0.13168,0.019332,-0.078192,-0.103022,-0.050914,0.001833,0.028097,-0.071538,0.00272,0.008438,-0.24002,-0.284457,0.052573,-0.018513,0.143768,0.010605,0.31638,0.269419,-0.016737,0.10023,-0.154745,-0.076715,0.047223,0.257837,-0.070267,0.178728,-0.107351,-0.166363,0.092161,-0.091461,-0.096295,-0.231899,0.051272,0.09972,-0.21434,-0.110883,0.056796


In [0]:
Y.head()

1782    1
3414    1
2744    0
77      1
354     1
Name: label, dtype: int64

In [0]:
Y_train= pd.DataFrame(Y)

In [0]:
Y_train.head(1)

,label
1782,1


In [0]:
#Keras model

In [0]:
import tensorflow as tf

In [0]:
#for tensorflow 2.0, use tf.keras rather than keras

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import optimizers


In [0]:
model = Sequential()

In [0]:
#stacking layers on model, from keras documentation

model.add(Dense(16, activation='relu', input_dim=X_train.shape[1])) #earlier 16 was units = 10
model.add(Dropout(0.2)) #Dropout is a regularization technique in Keras #https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/ #prevents overfitting,
#Droput = 0.2 is OK for smaller NNs, 0.5 for bigger models,
model.add(Dense(1, activation='sigmoid')) #no of units = no. or columns in y train, earlier 1 was units = 1

#use sigmoid rather than softmax activation

In [0]:
#compile the model

In [0]:
import tensorflow as tf
#from keras.callbacks import EarlyStopping
optimizer = tf.keras.optimizers.Adam(lr= 0.001) #default learning rate for keras, works well for most NN tasks. 
model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
# can use early stopping funtion here
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 16)                4816      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 4,833
Trainable params: 4,833
Non-trainable params: 0
_________________________________________________________________


In [0]:
X_train.shape, Y_train.shape

((3269, 300), (3269, 1))

In [0]:
#Val. data

X_val = df_val['text']
Y_val = df_val['label']
X_val_spacy = get_word_vectors(X_val)

In [0]:
X_val2 = pd.DataFrame(X_val_spacy)

In [0]:
Y_val2= pd.DataFrame(Y_val)

In [0]:
history = model.fit(X_train, Y_train, epochs=24, verbose=True, validation_data=(X_val2, Y_val2), batch_size=10)

#start with 100-200 epochs then tune after viewing the losses and accuracy plot
#increasing batch size may increase accuracy

Epoch 1/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3855 - accuracy: 0.7963 - val_loss: 0.5501 - val_accuracy: 0.7372
Epoch 2/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3860 - accuracy: 0.7944 - val_loss: 0.5679 - val_accuracy: 0.7347
Epoch 3/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3765 - accuracy: 0.8012 - val_loss: 0.5645 - val_accuracy: 0.7445
Epoch 4/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3833 - accuracy: 0.7859 - val_loss: 0.5564 - val_accuracy: 0.7347
Epoch 5/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3908 - accuracy: 0.7871 - val_loss: 0.5580 - val_accuracy: 0.7359
Epoch 6/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3799 - accuracy: 0.7947 - val_loss: 0.5535 - val_accuracy: 0.7372
Epoch 7/24
327/327 [==============================] - 1s 3ms/step - loss: 0.3815 - accuracy: 0.7914 - val_loss: 0.5517 - val_accuracy: 0.7433
Epoch 

In [0]:
#Val loss increases after epoch 24, so can use that as cut off

In [0]:
train_loss, train_accuracy = model.evaluate(X_train, Y_train, verbose=False)

In [0]:
print("Training Accuracy: {:.4f}".format(train_accuracy))

Training Accuracy: 0.8134


In [0]:
val_loss, val_accuracy = model.evaluate(X_val2, Y_val2, verbose=False)

In [0]:
print("Validation Accuracy: {:.4f}".format(val_accuracy))

Validation Accuracy: 0.7396


In [0]:
#Hypertune by learning rate, no of epochs, batch size etc.
